In [119]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/trains/test.csv
/kaggle/input/trains/train.csv


In [120]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from category_encoders import CountEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [121]:
import pandas as pd
test = pd.read_csv("../input/trains/test.csv")
train = pd.read_csv("../input/trains/train.csv")

In [122]:
dfCheto = pd.concat([train,test], sort=False)
dfCheto['precio'] = SimpleImputer(strategy='median').fit_transform(dfCheto[['precio']])
df = dfCheto.copy()

In [123]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [124]:
df.provincia.fillna('provinciaVacia', inplace = True)

In [125]:
aux2 = dfCheto[['ciudad', 'provincia']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby(['provincia']).agg(lambda x:x.provincia.value_counts().index[0]).reset_index().set_index('ciudad')
ciuds = aux2.T.to_dict('records').copy()
ciuds = ciuds[0]
df.ciudad.fillna(df.provincia.map(ciuds), inplace = True)

In [126]:
df.ciudad.fillna('ciudadVacia', inplace=True)

In [127]:
df.tipodepropiedad.fillna('tipoVacio', inplace=True)

In [128]:
aux2 = dfCheto[['idzona', 'ciudad']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby('ciudad').agg({'idzona':'median'})
ids = aux2.T.to_dict('records').copy()
ids = ids[0]
df.idzona.fillna(df.ciudad.map(ids), inplace = True)

In [129]:
df['idzona'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['idzona']])

In [130]:
aux = dfCheto[['lat', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lat':'mean'})
lats = aux.T.to_dict('records').copy()
lats = lats[0]
df.lat.fillna(df.ciudad.map(lats), inplace = True)

In [131]:
df['lat'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lat']])

In [132]:
aux = dfCheto[['lng', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lng':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.lng.fillna(df.ciudad.map(lngs), inplace = True)

In [133]:
df['lng'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lng']])

In [134]:
aux = dfCheto[['tipodepropiedad', 'habitaciones']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'habitaciones':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.habitaciones.fillna(df.tipodepropiedad.map(habts), inplace = True)

In [135]:
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])

In [136]:
aux = dfCheto[['tipodepropiedad', 'garages']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'garages':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.garages.fillna(df.tipodepropiedad.map(habts), inplace = True)

In [137]:
df.garages.fillna(0, inplace=True)

In [138]:
df.drop(columns=['direccion'], inplace=True)

In [139]:
aux = dfCheto[['antiguedad', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'antiguedad':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.antiguedad.fillna(df.ciudad.map(lngs), inplace = True)

In [140]:
df['antiguedad'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['antiguedad']])

In [141]:
aux = dfCheto[['banos', 'tipodepropiedad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'banos':'median'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.banos.fillna(df.tipodepropiedad.map(lngs), inplace = True)

In [142]:
df.banos.fillna(1, inplace=True)

In [143]:
df.titulo.fillna('', inplace=True)
df.descripcion.fillna('', inplace=True)
df['texto'] = df.titulo + ' ' + df.descripcion
df['texto'] = df.texto.str.len()
df.drop(columns = ['descripcion', 'titulo'], inplace = True)

In [144]:
aux = df.copy()

In [145]:
aux.metrostotales.fillna(0, inplace=True)
aux.metroscubiertos.fillna(0, inplace =True)
aux['metrostotalesCorregidos'] = aux[['metrostotales', 'metroscubiertos']].max(axis=1)
aux['metroscubiertosCorregidos'] = aux[['metrostotales', 'metroscubiertos']].min(axis=1)
df['metrostotales'] = aux['metrostotalesCorregidos']
df['metroscubiertos'] = aux['metroscubiertosCorregidos']

In [146]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['dia'] = df.fecha.dt.day
df['mes'] = df.fecha.dt.month
df['anio'] = df.fecha.dt.year
df.drop(columns=['fecha'], inplace=True)

In [147]:
df['publicacionesPorCiudad'] = CountEncoder().fit_transform(df['ciudad'])

In [148]:
df['publicacionesPorProvincia'] = CountEncoder().fit_transform(df['provincia'])

In [149]:
df['publicacionesPorTipo'] = CountEncoder().fit_transform(df['tipodepropiedad'])

In [150]:
transformador_categorias = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

transfomador_numero = Pipeline(steps=[('a', SimpleImputer(strategy = 'median'))])

In [151]:
df.columns

Index(['id', 'tipodepropiedad', 'ciudad', 'provincia', 'antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'precio', 'texto',
       'dia', 'mes', 'anio', 'publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo'],
      dtype='object')

In [152]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero, ['id', 'antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'texto',
       'dia', 'mes', 'anio', 'publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo']),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])

In [153]:

model = xgb.XGBRegressor(learning_rate= 0.1,n_estimators= 2000,scale_pos_weight= 2,max_depth= 5,min_child_weight= 5,gamma= 0.0,colsample_bytree= 0.75,
                     subsample= 0.7,colsample_bylevel= 0.65, tree_method='gpu_hist', gpu_id=0)

In [154]:
entrenamiento = df.head(240000).copy()

In [155]:
prueba = df.tail(60000).copy()

In [156]:
prueba.drop(columns=['precio'], inplace=True)

In [157]:
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()
#train_x, test_x, train_y, test_y = train_test_split(train_x, train_y,train_size=0.8, test_size=0.2, random_state = 0)

In [158]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [159]:
my_pipeline.fit(train_x, train_y)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[00:04:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('a',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                   

In [162]:
preds = my_pipeline.predict(prueba)

In [161]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(preds, train_y)

513600.04758341395

In [ ]:
res = pd.DataFrame(preds, index=prueba.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# RMSLE=1.0249284784393988 ?


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(res)